In [229]:
import pandas as pd

In [230]:
data=pd.read_csv("C:/House_Price_Prediction/notebooks/data/Housing.csv")

In [231]:
data.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [232]:
data.columns

Index(['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'mainroad',
       'guestroom', 'basement', 'hotwaterheating', 'airconditioning',
       'parking', 'prefarea', 'furnishingstatus'],
      dtype='object')

In [233]:
y=data['price']

In [234]:
cat_column=data.columns[data.dtypes=='O']

In [235]:
data_filtered = data[(data['price'] >= lower_bound) & (data['price'] <= upper_bound)]

In [236]:
median_value = data['price'].median()
data['price'] = np.where((data['price'] < lower_bound) | (data['price'] > upper_bound), median_value, data['price'])

In [237]:
X=data.drop(labels=['price'],axis=True)

In [238]:
y=data['price']

In [239]:
num_column=X.columns[X.dtypes!='O']

In [240]:
cat_column

Index(['mainroad', 'guestroom', 'basement', 'hotwaterheating',
       'airconditioning', 'prefarea', 'furnishingstatus'],
      dtype='object')

In [241]:
num_column

Index(['area', 'bedrooms', 'bathrooms', 'stories', 'parking'], dtype='object')

In [242]:
mainroad=['no','yes']
guestroom=['no','yes']
basement=['no','yes']
hotwaterheating=['no','yes']
airconditioning=['no','yes']
prefarea=['no','yes']
furnishingstatus=['unfurnished','semi-furnished','furnished']

In [243]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [244]:
num_pipeline=Pipeline(
    
    steps=[
        ('imputer',SimpleImputer()),
        ('scaling',StandardScaler())
    ]
)

In [245]:
cat_pipeline=Pipeline(
    
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('encoding',OrdinalEncoder(categories=[mainroad,basement,guestroom,airconditioning,
                                               hotwaterheating,prefarea,furnishingstatus]))
    ]
)

In [246]:
preprocessor=ColumnTransformer(
[
    ('num',num_pipeline,num_column),
    ('cat',cat_pipeline,cat_column)
]

)

In [337]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.20,random_state=21)

In [338]:
preprocessor.fit_transform(X_train)

array([[ 0.06120221,  2.75776416,  1.39300872, ...,  0.        ,
         0.        ,  1.        ],
       [-0.73671307, -1.32272851, -0.56619064, ...,  0.        ,
         0.        ,  0.        ],
       [-0.31428733,  1.39759994,  1.39300872, ...,  0.        ,
         0.        ,  2.        ],
       ...,
       [ 0.22078527, -1.32272851, -0.56619064, ...,  0.        ,
         0.        ,  1.        ],
       [-0.40815972,  0.03743571,  1.39300872, ...,  0.        ,
         0.        ,  0.        ],
       [-1.01833023,  0.03743571, -0.56619064, ...,  1.        ,
         0.        ,  1.        ]])

In [339]:
preprocessor.transform(X_test)

array([[ 0.1550746 ,  0.03743571,  1.39300872, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.41791728, -1.32272851, -0.56619064, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.8591175 ,  0.03743571,  1.39300872, ...,  1.        ,
         0.        ,  2.        ],
       ...,
       [-0.1359298 ,  0.03743571, -0.56619064, ...,  0.        ,
         1.        ,  0.        ],
       [-0.5489683 ,  0.03743571, -0.56619064, ...,  0.        ,
         0.        ,  2.        ],
       [ 0.74177702, -1.32272851, -0.56619064, ...,  0.        ,
         1.        ,  2.        ]])

In [340]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [341]:
X_train.head()

,num__area,num__bedrooms,num__bathrooms,num__stories,num__parking,cat__mainroad,cat__guestroom,cat__basement,cat__hotwaterheating,cat__airconditioning,cat__prefarea,cat__furnishingstatus
0,0.061202,2.757764,1.393009,0.188083,-0.776376,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.736713,-1.322729,-0.566191,-0.950862,-0.776376,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.314287,1.397600,1.393009,0.188083,1.542117,1.0,0.0,1.0,0.0,0.0,0.0,2.0
3,-0.196947,-1.322729,-0.566191,-0.950862,-0.776376,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.647905,0.037436,-0.566191,0.188083,-0.776376,1.0,0.0,1.0,0.0,1.0,1.0,2.0


In [342]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import numpy as np

In [343]:
def model_evaluate(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rsme=np.sqrt(mean_squared_error(true,predicted))
    r_square=r2_score(true,predicted)

    return mae,mse,rsme,r_square    

In [344]:
models={
    'LinearRegression': LinearRegression(),
    'ridge':Ridge(),
    'Lasso':Lasso(),
    'ElasticNet':ElasticNetCV(),
}

In [345]:
model_list=[]
r2_list=[]

In [346]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    
    #model prediction
    y_pred=model.predict(X_test)
    
    mae,mse,rsme,r_square=model_evaluate(y_test,y_pred)
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('model training performance')
    print("mean_absolute_error : ",mae)
    print('mean_squared_error : ',mse)
    print("rmse : ",rsme)
    print('r_square : ',r_square*100)
    
    r2_list.append(r_square)

    print('-'*30)
    print('\n')

LinearRegression
model training performance
mean_absolute_error :  761071.7293453823
mean_squared_error :  1039345844641.8333
rmse :  1019483.1262173167
r_square :  63.8794137287231
------------------------------


ridge
model training performance
mean_absolute_error :  761551.4353651142
mean_squared_error :  1042249342008.6997
rmse :  1020906.1377074289
r_square :  63.778507925646096
------------------------------


Lasso
model training performance
mean_absolute_error :  761071.8415891904
mean_squared_error :  1039347903016.7335
rmse :  1019484.1357356835
r_square :  63.87934219362384
------------------------------


ElasticNet
model training performance
mean_absolute_error :  1323622.8726060267
mean_squared_error :  2881551200086.3354
rmse :  1697513.240032706
r_square :  -0.14310371701911073
------------------------------


